In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import os

xla_flags = os.environ.get("XLA_FLAGS", "")
xla_flags += " --xla_gpu_triton_gemm_any=True"
## For deterministic results.
# xla_flags += " --xla_gpu_deterministic_ops=true --xla_gpu_autotune_level=0"
os.environ["XLA_FLAGS"] = xla_flags
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["MUJOCO_GL"] = "egl"
## In case your machine has GPUs of different types (pjit does not work with mixed GPUs).
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import functools
import json
from datetime import datetime

import jax
import jax.numpy as jp
import matplotlib.pyplot as plt
import mediapy as media
import mujoco
import numpy as np
import wandb
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import train as ppo
from etils import epath
from flax.training import orbax_utils
from IPython.display import clear_output, display
from orbax import checkpoint as ocp

from mujoco_playground import BraxEnvWrapper, locomotion
from mujoco_playground.locomotion.h1 import randomize as h1_randomize
from mujoco_playground.locomotion.spot import randomize as spot_randomize

jax.config.update("jax_compilation_cache_dir", "/tmp/jax_cache")
jax.config.update("jax_persistent_cache_min_entry_size_bytes", -1)
jax.config.update("jax_persistent_cache_min_compile_time_secs", 0)

RANDOMIZER = {
    "SpotInplaceGaitTracking": spot_randomize,
    "H1InplaceGaitTracking": h1_randomize,
}

In [ ]:
env_name = "SpotInplaceGaitTracking"
env_cfg = locomotion.get_default_config(env_name)
env_cfg

In [5]:
# Setup wandb logging.
USE_WANDB = False

if USE_WANDB:
  wandb.init(project="mjxrl", config=env_cfg)
  wandb.config.update({
      "env_name": env_name,
  })

In [ ]:
SUFFIX = None
FINETUNE_PATH = None

# Generate unique experiment name.
now = datetime.now()
timestamp = now.strftime("%Y%m%d-%H%M%S")
exp_name = f"{env_name}-{timestamp}"
if SUFFIX is not None:
  exp_name += f"-{SUFFIX}"
print(f"Experiment name: {exp_name}")

# Possibly restore from the latest checkpoint.
if FINETUNE_PATH is not None:
  FINETUNE_PATH = epath.Path(FINETUNE_PATH)
  latest_ckpts = list(FINETUNE_PATH.glob("*"))
  latest_ckpts.sort()
  latest_ckpt = latest_ckpts[0]
  restore_checkpoint_path = latest_ckpt
  print(f"Restoring from: {restore_checkpoint_path}")
else:
  restore_checkpoint_path = None

In [ ]:
ckpt_path = epath.Path("checkpoints").resolve() / exp_name
ckpt_path.mkdir(parents=True, exist_ok=True)
print(f"Checkpoint path: {ckpt_path}")

with open(ckpt_path / "config.json", "w") as fp:
  json.dump(env_cfg.to_dict(), fp, indent=4)


def policy_params_fn(current_step, make_policy, params):
  orbax_checkpointer = ocp.PyTreeCheckpointer()
  save_args = orbax_utils.save_args_from_target(params)
  path = ckpt_path / f"{current_step}"
  orbax_checkpointer.save(path, params, force=True, save_args=save_args)


make_networks_factory = functools.partial(
    ppo_networks.make_ppo_networks,
    policy_hidden_layer_sizes=(128, 128, 128, 128),
)

train_fn = functools.partial(
    ppo.train,
    num_timesteps=70_000_000,
    num_evals=5,
    reward_scaling=1.0,
    episode_length=env_cfg.episode_length,
    normalize_observations=True,
    action_repeat=1,
    unroll_length=20,
    num_minibatches=32,
    num_updates_per_batch=4,
    discounting=0.97,
    learning_rate=3e-4,
    entropy_cost=1e-2,
    num_envs=8192,
    batch_size=256,
    network_factory=make_networks_factory,
    policy_params_fn=policy_params_fn,
    randomization_fn=RANDOMIZER[env_name].domain_randomize,
    seed=1,
    restore_checkpoint_path=restore_checkpoint_path,
    max_grad_norm=1.0,
)

In [ ]:
x_data, y_data, y_dataerr = [], [], []
times = [datetime.now()]


def progress(num_steps, metrics):
  # Log to wandb.
  if USE_WANDB:
    wandb.log(metrics, step=num_steps)

  # Plot.
  clear_output(wait=True)
  times.append(datetime.now())
  x_data.append(num_steps)
  y_data.append(metrics["eval/episode_reward"])
  y_dataerr.append(metrics["eval/episode_reward_std"])

  plt.xlim([0, train_fn.keywords["num_timesteps"] * 1.25])
  plt.ylim([0, 75])
  plt.xlabel("# environment steps")
  plt.ylabel("reward per episode")
  plt.title(f"y={y_data[-1]:.3f}")
  plt.errorbar(x_data, y_data, yerr=y_dataerr, color="blue")

  display(plt.gcf())


env = locomotion.load(env_name, config=env_cfg)
eval_env = locomotion.load(env_name, config=env_cfg)
make_inference_fn, params, _ = train_fn(
    environment=BraxEnvWrapper(env),
    progress_fn=progress,
    eval_env=BraxEnvWrapper(eval_env),
)
if len(times) > 1:
  print(f"time to jit: {times[1] - times[0]}")
  print(f"time to train: {times[-1] - times[1]}")

In [9]:
inference_fn = make_inference_fn(params, deterministic=True)
jit_inference_fn = jax.jit(inference_fn)

In [10]:
eval_env = locomotion.load(env_name, config=env_cfg)
jit_reset = jax.jit(eval_env.reset)
jit_step = jax.jit(eval_env.step)

In [12]:
# from mujoco_playground.locomotion.go1.visualization import draw_joystick_command
# x_min = 0.0
# x_max = 1.0
# def get_scl(x: float) -> float:
# return (x - x_min) / (x_max - x_min)
# yaw = 0.2
# x = 0.1
# cmd = jp.array([x, 0, yaw])

rng = jax.random.PRNGKey(12345)
rng, reset_rng = jax.random.split(rng)
state = jit_reset(reset_rng)
# state.info["command"] = cmd

# Change gait type.
PHASES = jp.array([
    [0, jp.pi, jp.pi, 0],  # trot
    [0, 0.5 * jp.pi, jp.pi, 1.5 * jp.pi],  # walk
    [0, jp.pi, 0, jp.pi],  # pace
    [0, 0, jp.pi, jp.pi],  # bound
    [0, 0, 0, 0],  # pronk
])
# PHASES = jp.array(
#     [
#         [0, jp.pi],  # walk
#         [0, 0],  # jump
#     ]
# )
gait = jp.array(0)
state.info["gait"] = gait
state.info["phase"] = PHASES[gait]

modify_scene_fns = []
rollout = []
swing_peak = []
foot_height = 0.4  # [0.08, 0.4]
gait_freq = 2  # [0.5, 4.0]
phase_dt = 2 * jp.pi * env.dt * jp.array(gait_freq)
state.info["phase_dt"] = phase_dt
state.info["gait_freq"] = jp.array(gait_freq)
for i in range(800):
  if i % 200 == 0:
    foot_height = np.random.uniform(0.08, 0.4)
    state.info["foot_height"] = jp.array(foot_height)
    # gait_freq = np.random.uniform(1.5, 4.0)
    # phase_dt = 2 * jp.pi * env.dt * jp.array(gait_freq)
    # state.info["phase_dt"] = phase_dt
    # state.info["gait_freq"] = jp.array(gait_freq)
    # x += 0.2
    # yaw = np.random.uniform(-0.7, 0.7)
    # cmd = jp.array([x, 0, yaw])
    # state.info["command"] = cmd
    # Print all the sampled parameters.
    # print(
    #     f"foot_height: {foot_height:.3f}, gait_freq: {gait_freq:.3f}, phase_dt: {phase_dt:.3f}, yaw: {yaw:.3f}, x: {x:.3f}"
    # )

  act_rng, rng = jax.random.split(rng)
  ctrl, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_step(state, ctrl)
  # state.info["command"] = cmd
  state.info["phase_dt"] = phase_dt
  state.info["gait_freq"] = jp.array(gait_freq)
  rollout.append(state)
  swing_peak.append(state.info["swing_peak"])

  # xyz = np.array(state.data.xpos[env._torso_body_id])
  # xyz += np.array([0, 0, 0.2])
  # x_axis = state.data.xmat[env._torso_body_id, 0]
  # yaw = -np.arctan2(x_axis[1], x_axis[0])
  # modify_scene_fns.append(
  #     functools.partial(
  #         draw_joystick_command,
  #         cmd=cmd,
  #         xyz=xyz,
  #         foot_height=foot_height,
  #         foot_pos=state.data.site_xpos[env._feet_site_id],
  #         theta=yaw,
  #         scl=get_scl(x),
  #     )
  # )

In [ ]:
render_every = 1
fps = 1.0 / eval_env.dt / render_every
print(f"fps: {fps}")

traj = rollout[::render_every]

scene_option = mujoco.MjvOption()
scene_option.geomgroup[2] = True
scene_option.geomgroup[3] = False
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True

frames = eval_env.render(
    traj,
    camera="side",
    scene_option=scene_option,
    height=480 * 2,
    width=640 * 2,
)
media.show_video(frames, fps=fps)
# media.write_video("h1_inplace_gait_tracking.mp4", frames, fps=fps, qp=18)